In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import regex

# Import label encoder
from sklearn.preprocessing import LabelEncoder

# Import credentials for spotify
import cred

# Set pandas to print out all columns
pd.set_option('display.max_columns', None)

In [2]:
playlist = pd.read_csv('/Users/CurtisIrvine/Downloads/VTSS.csv', on_bad_lines='skip')

In [3]:
# Score the dates between 0 and 100 

# Order the data frame descending by album date
playlist = playlist.sort_values(by=['Album Date'], ascending=False)

# Encode the album date column
playlist['album_date_encoded'] = LabelEncoder().fit_transform(playlist['Album Date'])

# Normalise the album date encoded field between 0 and 100
playlist['album_date_encoded'] = playlist['album_date_encoded'] / playlist['album_date_encoded'].max() * 100

In [4]:
# Normalise the # field between 0 and 100
playlist['Intuition'] = playlist['#'] / playlist['#'].max() * 100

# Map 0 to 100 and 100 to 0 
playlist['Intuition'] = playlist['Intuition'].map(lambda x: 100 - x)

In [5]:
# Convert Time field from object of MM:SS to seconds
playlist['Time'] = playlist['Time'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

In [6]:
# Map BPM between 0 and 100
playlist['BPM'] = (playlist['BPM'] / playlist['BPM'].max()) * 100

In [7]:
playlist['Speechiness'] = (playlist['Speechiness'] / playlist['Speechiness'].max()) * 100
# Map 0 to 100 and 100 to 0 
playlist['Speechiness'] = playlist['Speechiness'].map(lambda x: 100 - x)

In [8]:
playlist['Instrumentalness'] = (playlist['Instrumentalness'] / playlist['Instrumentalness'].max()) * 100

In [9]:
# Make composite wordiness metric 
playlist['Beat'] = (playlist['Speechiness'] + playlist['Instrumentalness']) / 2

In [10]:
playlist.columns

Index(['#', 'Song', 'Artist', 'Popularity', 'BPM', 'Genres', 'Parent Genres',
       'Album', 'Album Date', 'Time', 'Danceability', 'Energy', 'Acousticness',
       'Instrumentalness', 'Happiness', 'Speechiness', 'Liveness', 'Loudness',
       'Key', 'Time Signature', 'Added At', 'Spotify Track Id', 'Album Label',
       'Camelot', 'Spotify Track Img', 'Song Preview', 'album_date_encoded',
       'Intuition', 'Beat'],
      dtype='object')

In [11]:
# Create a composite metric to order the data frame by
composite_columns = [
    'BPM',
    'Danceability',
    'Energy',
    'Energy',
    'Intuition',
    # 'Intuition',
    # 'Intuition',
]

# Add together the values of composite_columns in the playlist dataframe and divide by the number of columns
playlist['composite_metric'] = playlist[composite_columns].sum(axis=1)/len(composite_columns)

In [12]:
# Order the data frame by the composite metric
playlist = playlist.sort_values(by=['composite_metric'], ascending=False)

In [13]:
playlist[['Song', 'Artist', 'composite_metric']]

,Song,Artist,composite_metric
1,Banana Brain,Die Antwoord,86.148936
12,Atmosféra,Brutalismus 3000,81.968085
6,Slow It Down,Skin On Skin,81.721277
8,Don't Fuck Around,"Skin On Skin,X CLUB.",81.370213
11,Parking Lot - VIGILE Remix,"Shawn Cartier,VIGILE",80.893617
2,Hey Boy Hey Girl,The Chemical Brothers,80.823404
0,Perfect (Exceeder),"Mason,Princess Superstar",79.774468
10,The Age Of Love (Charlotte de Witte & Enrico S...,"Age Of Love,Charlotte de Witte,Enrico Sangiuliano",79.719149
15,DIP N DRIP,COBRAH,79.191489
3,Happy Go Sucky Fucky,Die Antwoord,79.097872


In [20]:
len(playlist)

40

In [15]:
# Number of songs considered peak
peak_number = 40

# Store the last 150 entries in a second dataframe
low_songs = playlist.iloc[-(len(playlist) - peak_number):]

# Drop these from playlist
playlist = playlist.drop(low_songs.index)

# Sort playlist by composite metric ascending
playlist = playlist.sort_values(by=['composite_metric'], ascending=True)

# Randomly select songs from the low_songs dataframe
first_songs = low_songs.sample(n=int(len(low_songs)/2))

# Order first_songs by the composite metric
first_songs = first_songs.sort_values(by=['composite_metric'], ascending=True)

# Drop these from low_songs
final_songs = low_songs.drop(first_songs.index)

# Order final_songs by the composite metric
final_songs = final_songs.sort_values(by=['composite_metric'], ascending=False)

In [16]:
# Find time of first songs
ramp_up = first_songs['Time'].sum()

# Return as hours 
print('Ramp up time is {} hours'.format(ramp_up / 3600))

Ramp up time is 1.4458333333333333 hours


In [17]:
# Concatenate first_songs to the start of playlist
playlist = pd.concat([first_songs, playlist])

# Concatenate final_songs to the end of playlist
playlist = pd.concat([playlist, final_songs])

In [19]:
playlist[['Song', 'Artist', 'composite_metric']]

,Song,Artist,composite_metric
31,LETS GO ZANTE,"Babymorocco,River Moon",60.982979
30,Venus Fly,"Grimes,Janelle Monáe",61.608511
28,4ÆM,Grimes,62.959574
24,Scream,Newton,63.861702
39,I Felt Love (Salzbauer Rave Edit),"Blue Hawaii,Salzbauer",64.378723
42,Omen,The Prodigy,64.702128
40,It's A Killa,"FISHER,Shermanology",67.153191
25,Fear,Hensonn,68.236170
37,Voodoo People - Pendulum Mix,"The Prodigy,Pendulum",68.429787
27,Kill V. Maim,Grimes,71.085106


In [22]:
playlist

,#,Song,Artist,Popularity,BPM,Genres,Parent Genres,Album,Album Date,Time,Danceability,Energy,Acousticness,Instrumentalness,Happiness,Speechiness,Liveness,Loudness,Key,Time Signature,Added At,Spotify Track Id,Album Label,Camelot,Spotify Track Img,Song Preview,album_date_encoded,Intuition,Beat,composite_metric
31,32,LETS GO ZANTE,"Babymorocco,River Moon",17,62.0,NaN,NaN,LETS GO ZANTE,2022-09-09,266,79,66,2,0.000000,83,50.0,10,-10,F#/G♭ Major,4,2023-01-18,3epQlgSd5THLImbQk9QLOV,PhatBoy,2B,https://p.scdn.co/mp3-preview/057c14018ac25070...,https://i.scdn.co/image/ab67616d000048515efbc6...,86.486486,31.914894,25.000000,60.982979
30,31,Venus Fly,"Grimes,Janelle Monáe",49,64.0,"art pop, canadian electropop, dance pop, elect...","R&B, Pop, Hip Hop, Dance/Electronic",Art Angels,2015-11-06,225,62,74,0,90.625000,25,100.0,30,-7,E Minor,4,2023-01-16,1QxbBB80IuPwhwW1ygGfPh,4AD,9A,https://p.scdn.co/mp3-preview/c407834fcf684e6e...,https://i.scdn.co/image/ab67616d000048514f1235...,16.216216,34.042553,95.312500,61.608511
28,29,4ÆM,Grimes,58,46.5,"art pop, canadian electropop, dance pop, elect...","Pop, Dance/Electronic",Miss Anthropocene (Deluxe Edition),2020-02-21,270,48,91,1,30.208333,12,100.0,20,-4,G Minor,4,2023-01-15,4keclC0eBhQm8lFtYmX01K,4AD,6A,https://p.scdn.co/mp3-preview/866f65e397951b0f...,https://i.scdn.co/image/ab67616d000048511a302a...,27.027027,38.297872,65.104167,62.959574
24,25,Scream,Newton,32,77.5,NaN,NaN,Scream,2022-11-11,190,55,70,11,97.916667,20,25.0,20,-12,G Major,4,2023-01-15,1ISTY6R9bmXxHuLeIOvwnY,Newton,9B,https://p.scdn.co/mp3-preview/dc0b659f90a8e2c9...,https://i.scdn.co/image/ab67616d000048513f317f...,94.594595,46.808511,61.458333,63.861702
39,40,I Felt Love (Salzbauer Rave Edit),"Blue Hawaii,Salzbauer",37,76.0,"art pop, chillwave, quebec indie,","Pop, Rock",I Felt Love (Salzbauer Rave Edit),2022-12-06,279,65,83,0,84.375000,71,75.0,0,-9,C♯/D♭ Major,4,2023-02-14,5FoW2bf8VVfjYMnOWOAgen,Arbutus Records,3B,https://p.scdn.co/mp3-preview/c423ba36f43f7d41...,https://i.scdn.co/image/ab67616d000048513862b7...,97.297297,14.893617,79.687500,64.378723
42,43,Omen,The Prodigy,56,70.0,"big beat, breakbeat, dance rock, hardcore tech...","Dance/Electronic, Rock, Pop",Invaders Must Die,2009-02-23,216,55,95,0,12.500000,56,100.0,20,-5,G Major,4,2023-02-18,144adL7pGHEWRwute2wxzZ,Take Me To The Hospital,9B,https://p.scdn.co/mp3-preview/63c8c22fbeac9722...,https://i.scdn.co/image/ab67616d00004851bce53a...,8.108108,8.510638,56.250000,64.702128
40,41,It's A Killa,"FISHER,Shermanology",45,63.0,"australian house, edm, dutch house, house",Dance/Electronic,It's A Killa,2022-03-11,200,78,91,0,1.041667,82,100.0,10,-5,G Major,4,2023-02-15,0bsdfTgzxOPN4RcZcGJJ3t,Good Company,9B,https://p.scdn.co/mp3-preview/a3d8db6ccde8af7b...,https://i.scdn.co/image/ab67616d000048510eaf32...,59.459459,12.765957,50.520833,67.153191
25,26,Fear,Hensonn,64,57.5,drift phonk,NaN,Fear,2022-09-30,131,69,85,49,89.583333,26,100.0,10,-6,G#/A♭ Major,4,2023-01-15,0MTBiPyJ4u3XoS1JKBktdb,Hensonn,4B,https://p.scdn.co/mp3-preview/3da47ab7f16b1836...,https://i.scdn.co/image/ab67616d00004851fa05b4...,89.189189,44.680851,94.791667,68.236170
37,38,Voodoo People - Pendulum Mix,"The Prodigy,Pendulum",50,87.0,"big beat, breakbeat, dance rock, hardcore tech...","Dance/Electronic, Rock, Pop",Their Law The Singles 1990 - 2005,2005-10-17,307,44,96,0,91.666667,53,100.0,10,-5,C♯/D♭ Major,4,2023-01-31,7C0PzxbsQx59DgGVRsquwJ,XL Recordings,3B,https://p.scdn.co/mp3-preview/3aa50ffcb3a96a57...,https://i.scdn.co/image/ab67616d0000485180a547...,2.702703,19.148936,95.833333,68.429787
27,28,Kill V. Maim,Grimes,61,67.0,"art pop, canadian electropop, dance pop, elect...","Pop, Dance/Electronic",Art Angels,2015-11-06,246,58,95,1,8.333333,46,100.0,10,-2,B Minor,4,2023-01-15,3WXhshrs1fzwF3rQE399Gq,4AD,10A,https://p.scdn.co/mp3-preview/0ca25a84ed18a819...,https://i.scdn.co/image/ab67616d000048514f1235...,16.216216,40.425532,54.166667,71.085106


In [587]:
# Reset the index of the playlist dataframe
playlist = playlist.reset_index()

# Find the index of the element with the highest composite metric
best_song_position = playlist['composite_metric'].idxmax()

# Sum the time up to the best song position
peak_time = playlist['Time'].iloc[:best_song_position].sum()

# Print the peak time as hours
print("The peak time is {} hours".format(peak_time / 3600))

The peak time is 5.825277777777778 hours


# Spotipy

In [588]:
# Scope to modify the playlist
scope = 'playlist-modify-public'

# Authenticate
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=cred.client_ID,
        client_secret=cred.client_SECRET,
        redirect_uri=cred.redirect_url,
        scope=scope
    )
)

In [589]:
playlist_order = playlist['#'].tolist()

In [590]:
items = playlist['Spotify Track Id'].tolist()
# Split into 100s 
items1 = items[:100]
items2 = items[100:200]
items3 = items[200:]

In [591]:
# Wipe Playlist
replace = sp.playlist_replace_items(cred.test_id,
                                    items=[])

In [592]:
# Add items
add = sp.playlist_add_items(cred.test_id,
                            items = items1
)
add = sp.playlist_add_items(cred.test_id,
                            items = items2
)
add = sp.playlist_add_items(cred.test_id,
                            items = items3
)